<a href="https://colab.research.google.com/github/ShoaibSheriff/Computer-Vision/blob/master/Action%20Spotting/Action_Spot_LSTM_Drink.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt

import torch.optim as optim
from torch.autograd import Variable
import torchvision.transforms as transforms

import torch
import torch.nn as nn

# import tensorflow as tf

from pprint import pprint

%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [9]:
import torchvision.models as models

resnet = models.resnet34(pretrained=True)
# rs = nn.Sequential(*list(resnet50.children())[:-1])
encoder = nn.Sequential(resnet, nn.Sequential(nn.Linear(1000, 96)))
encoder.eval()

import copy
gpu_dtype = torch.cuda.FloatTensor

encoder_gpu = copy.deepcopy(encoder)
encoder_gpu = encoder_gpu.cuda()
encoder_gpu.eval()

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 334MB/s]


Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [0]:
import glob, os
os.chdir("/content/drive/My Drive/CSE 527/Project/1fps/ava/frames/")
train_ids = []
for file in glob.glob("*.mp4"):
    train_ids.append(file[:-4])
# print(train_ids)

In [0]:
df_search_seq = pd.read_csv(r"/content/drive/My Drive/CSE 527/Project/1fps/ava/ava-human-searches_anno.csv")
df_search_seq

,search-id,step-id,time,video-id,target-action
0,0,0,849.623600,Ag-pXiLrd48,clink glass
1,0,1,852.323675,Ag-pXiLrd48,clink glass
2,0,2,855.023750,Ag-pXiLrd48,clink glass
3,0,3,850.523625,Ag-pXiLrd48,clink glass
4,0,4,853.223700,Ag-pXiLrd48,clink glass
...,...,...,...,...,...
893476,3987,56,726.326631,mfsbYdLx9wE,write
893477,3987,57,728.126697,mfsbYdLx9wE,write
893478,3987,58,732.626862,mfsbYdLx9wE,write
893479,3987,59,741.627192,mfsbYdLx9wE,write


In [0]:
df_drink = df_search_seq.loc[((df_search_seq['target-action'] == 'drink') & (df_search_seq['video-id'].isin(train_ids)))]

In [0]:
searchids = df_drink.groupby(['search-id']).agg(['count']).sort_values(by = [('time', 'count')], ascending=False).reset_index()['search-id'].to_list()[2:]
data = []

for searchid in searchids:
  # print(searchid)
  df_search = df_drink.loc[(df_drink['search-id'] == searchid)].sort_values(['step-id'])

  video_id = df_search.loc[(df_search['step-id'] == 0)]['video-id'].to_list()[0]
  # print(video_id)
  input_ts = df_search['time'].to_list()
  if (len(input_ts) < 8 or len(input_ts) > 1200):
    continue
  temp = input_ts[1:]
  temp.append(input_ts[-1])
  jumps = [i-j for i, j in zip(temp, input_ts)]

  video_f = '/content/drive/My Drive/CSE 527/Project/1fps/ava/videos/'+video_id+'.mp4'
  
  sample = {'video': video_f, 'input':input_ts, 'output':jumps}
  # print(input_ts)
  # print(jumps)self.hidden
  data.append(sample)

In [0]:
from tqdm import tqdm

def generate_batches(instances, batch_size):
    """
    Generates and returns batch of tensorized instances in a chunk of batch_size.
    """

    def chunk(items, num):
        return [items[index:index+num] for index in range(0, len(items), num)]
    batches_of_instances = chunk(instances, batch_size)

    batches = []
    for batch_of_instances in tqdm(batches_of_instances):

        num_steps = [len(instance["input"])
                         for instance in batch_of_instances]
        max_steps = max(num_steps)

        count = min(batch_size, len(batch_of_instances))
        batch = {"input_seq": np.full((count, max_steps), -1, dtype=np.float32),
                 "output_steps": np.full((count, max_steps), -1, dtype=np.float32)}

        batch["video_path"] = np.full((count), "", dtype=object)

        for batch_index, instance in enumerate(batch_of_instances):
            num_tokens = len(instance["input"])

            inputs = np.array(instance["input"])
            batch["input_seq"][batch_index][:num_tokens] = inputs

            pos_inputs = np.array(instance["output"])
            batch["output_steps"][batch_index][:num_tokens] = pos_inputs

            # print(batch["video_path"].shape)
            batch["video_path"][batch_index] = instance["video"]

        batches.append(batch)
        # print('\n' + str(len(batches)))

    return batches

batches = generate_batches(data, 4)

100%|██████████| 61/61 [00:00<00:00, 4066.83it/s]


In [0]:
def train(model, optimizer, train_batches, num_epochs):

    train_batch_steps = [batch_inputs.pop("output_steps") for batch_inputs in train_batches]
    model.train()

    for epoch in range(num_epochs):
        print(f"\nEpoch{epoch}")

        epoch_loss = 0
        generator_tqdm = tqdm(list(zip(train_batches, train_batch_steps)))
        for batch_inputs, batch_steps in generator_tqdm:
            # with tf.GradientTape() as tape:
              ret = model(**batch_inputs, training=True)
              preds = ret['preds']
              slens = ret['seq_lens']
              mask = ret['mask']
              # print('in train')
              # print(preds.shape)
              # print(batch_steps.shape)
              # print(mask.shape)
              # print('mask', mask)
              losses = torch.nn.SmoothL1Loss( reduction = 'none')(preds, torch.from_numpy(batch_steps))
              # """.cuda()""")
              # print(losses)

              real_loss = torch.mul(losses, torch.from_numpy(mask))
              '''.cuda())'''
              # print('real_loss', real_loss)

              loss_val = torch.sum(real_loss)/ np.sum(mask)
              # print('loss', loss_val.data.item())

              epoch_loss += loss_val
              optimizer.zero_grad()
              loss_val.backward()
              optimizer.step()
              print('cuda memory', torch.cuda.memory_allocated())

        epoch_loss = epoch_loss / len(train_batches)
        print(f"Train loss for epoch: {epoch_loss}")
        torch.save(model.state_dict(), "/content/drive/My Drive/CSE 527/Project/model"+str(epoch)+'.v5.fltr.drp.h5')


In [0]:
class LSTMTSearch(nn.Module):

    def __init__(self):
        super(LSTMTSearch, self).__init__()

        self.hidden_size = 128
        self.batch_size = 4

        self.encoder = encoder_gpu
        # self.encoder = encoder
        self.lstm = nn.LSTM(batch_first = True, input_size = 97, hidden_size = self.hidden_size, dropout = 0.25, num_layers = 2)
        self.linear = nn.Linear(self.hidden_size, 1)

    def get_init_hidden_states(self):
        return (torch.zeros(2, self.batch_size, self.hidden_size),
                torch.zeros(2, self.batch_size, self.hidden_size))

    def forward(self, input_seq, video_path, training):

        # print(input_seq[0])
        # print(video_path[0])

        self.lstm.hidden = self.get_init_hidden_states()

        batch_ips = None
        seq_lengths = np.zeros((len(video_path)))

        for idx, video in enumerate(video_path):
          print('cuda memory', torch.cuda.memory_allocated())
          cap = cv2.VideoCapture(video)

          images = None
          # print(len(input_seq[idx]))
          slen = 0
          to_add = 0

          for ts in input_seq[idx]:
            
            if ts > -1:
              # print(ts)

              slen += 1

              cap.set(cv2.CAP_PROP_POS_MSEC, (904+ts)*1000) 
              _, frame = cap.read()
              # print(video)
              # plt.imshow(frame)

              train_transforms = transforms.Compose([
              transforms.ToPILImage(),
              transforms.Resize(70),
              transforms.CenterCrop(64),
              transforms.ToTensor(),
              transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
              ])
            
              frame2 = train_transforms(frame).cuda()
              frame2.requires_grad=False

              # plt.imshow(frame2)
              # plt.show()

              if images is None:
                images = frame2.unsqueeze(0)
              else:
                images = torch.cat((images, frame2.unsqueeze(0)), dim=0)

              # del frame2
            else:
              to_add += 1
            #     images = torch.cat((images, zeros), dim=0)
          # print(images.shape)

          seq_lengths[idx] = slen
          # print('actual sequence length', slen)

          activations = self.encoder(images).cpu()
          # print('after image encode ', activations.shape)

          # print('cuda memory', torch.cuda.memory_allocated())

          zeros_to_pad = torch.zeros([to_add, 96], dtype=torch.float32)
          """.cuda()"""
          padded_activations = torch.cat((activations, zeros_to_pad), dim=0)
          # print('after padding', padded_activations.shape)

          # print(torch.from_numpy(input_seq[idx]).unsqueeze(1).shape)

          padded_activations = torch.cat((padded_activations, torch.from_numpy(input_seq[idx]).unsqueeze(1)), dim = 1)
          """.cuda()"""
          # ), dim = 1)
          # print('after add timestamp ', padded_activations.shape)

          if batch_ips is None:
            batch_ips = padded_activations.unsqueeze(0)
          else:
            batch_ips = torch.cat((batch_ips, padded_activations.unsqueeze(0)), dim=0)

            # del padded_activations
            # del activations
            # del images
            # del zeros_to_pad
            # torch.cuda.empty_cache()

        print(batch_ips.shape)
        # print(seq_lengths)

        padded_batch_ips = torch.nn.utils.rnn.pack_padded_sequence(batch_ips, seq_lengths, batch_first=True)

        output, (h_n, c_n) = self.lstm(padded_batch_ips)
        # output1, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        output2, _ = torch.nn.utils.rnn.pad_packed_sequence(output, batch_first=True)
        # print('lstm out', output2.shape)
        # print(h_n.shape)
        # print(c_n.shape)

        # # preds1 = self.linear(output1.view(-1, output1.size(2)))
        # # print('preds', preds1.shape)

        preds2 = self.linear(output2)
        # print('preds', preds2.shape)

        ret = {}
        ret['preds'] = preds2.squeeze()
        ret['seq_lens'] = seq_lengths
        ret['mask'] = input_seq > -1

        return ret
# model = LSTMTSearch()
# train(model, optim.Adam(model.parameters(), lr=0.0001), batches, 10)

In [0]:
model = LSTMTSearch()
train(model, optim.Adam(model.parameters(), lr=0.0001), batches, 20)

  0%|          | 0/61 [00:00<?, ?it/s]


Epoch0
cuda memory 88635904
cuda memory 3234025984
cuda memory 6086322688
cuda memory 8932459008
torch.Size([4, 1192, 97])


  2%|▏         | 1/61 [05:34<5:34:22, 334.38s/it]

cuda memory 358729728
cuda memory 358729728
cuda memory 3060686848
cuda memory 5757376512
cuda memory 8454434816
torch.Size([4, 1025, 97])


  3%|▎         | 2/61 [11:59<5:43:51, 349.69s/it]

...

cuda memory 358729728
cuda memory 358729728
cuda memory 2589237248
cuda memory 4790671360
cuda memory 6992547840
torch.Size([4, 846, 97])


  7%|▋         | 4/61 [18:06<4:00:33, 253.23s/it]

cuda memory 358729728
cuda memory 358729728
cuda memory 2506039296
cuda memory 4533794816
cuda memory 6445551616
torch.Size([4, 809, 97])


  8%|▊         | 5/61 [20:30<3:25:46, 220.47s/it]

cuda memory 358729728
cuda memory 358729728


In [0]:
####################

# Test


In [10]:
model = LSTMTSearch()
# model = LSTMTSearch(hidden_size=128, batch_size=1)
model.load_state_dict(torch.load('/content/drive/My Drive/CSE 527/Project/model7.v5.fltr.drp.h5'))
model.eval()

LSTMTSearch(
  (encoder): Sequential(
    (0): ResNet(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNor

In [0]:
import glob, os
os.chdir("/content/drive/My Drive/CSE 527/Project/1fps/ava/test_videos")
test_ids = []
for file in glob.glob("*.mp4"):
    test_ids.append(file[:-4])

In [0]:
df_test = pd.read_csv(r"/content/drive/My Drive/CSE 527/Project/1fps/ava/ava_test_v1.0.csv",names=['video_id','middle_frame_timestamp', 'x1','y1','x2','y2', 'action_id'], header=None)

In [0]:
df_test_dr_valid = df_test.loc[((df_test['action_id'] == 27) & (df_test['video_id'].isin(test_ids)))]

In [0]:
MIN = 904
MAX = 1798

In [0]:
video_ids = df_test_dr_valid['video_id'].unique()

In [22]:
video_ids

array(['7YpF6DntOYw', 'BCiuXAuCKAU', 'HKjR70GCRPE', 'IzvOYVMltkI',
       'LgBQlW6OTr0', 'P60OxWahxBQ', 'g1wyIcLPbq0', 'kMy-6RtoOVU',
       'yn9WN9lsHRE', 'z-fsLpGHq6o', 'zR725veL-DI'], dtype=object)

In [0]:
def get_next_guess(cap, guess_time, hidden):
    
    cap.set(cv2.CAP_PROP_POS_MSEC,guess_time*1000) 
    ret, frame = cap.read()
    
    train_transforms = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(70),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
  
    frame2 = train_transforms(frame).cuda()
    frame2.requires_grad=False

    frame3 = frame2.unsqueeze(0)
    activations = model.encoder(frame3).cpu()
    # print(activations.shape)
    
    wrap = torch.from_numpy(numpy.array([[guess_time]], dtype = 'float32'))

    padded_activations = torch.cat((activations, wrap), dim = 1)
    # print(padded_activations.shape)

    batch_ip = padded_activations.unsqueeze(0)
    # print(batch_ip.shape)

    model.lstm.hidden = hidden
    output, (h_n, c_n) = model.lstm(batch_ip)
    # print(output.shape)

    preds = model.linear(output)
    print(preds)
        
    return (h_n, c_n), guess_time + preds.data.item()

In [0]:
import random

def find_action(dataf, video, hidden_states) :    
    
    search_num = 1
    guess_time = random.randint(MIN, MAX)
    
    cap = cv2.VideoCapture('/content/drive/My Drive/CSE 527/Project/1fps/ava/test_videos/'+video+'.mp4')

    for i in range(500) :
        a_g = np.rint(guess_time)
        print(a_g)
        if a_g in dataf:
            break;
        if (a_g - 1 in dataf) & (guess_time + 2 in dataf):
            break;
        if (a_g - 2 in dataf) & (guess_time + 1 in dataf):
            break;
        else :
            search_num = search_num + 1
        if (a_g <= MIN or a_g >= MAX):
            return -1
        
        hidden_states, guess_time = get_next_guess(cap, guess_time, hidden_states)
        if (guess_time == -1):
            return 500
    
    return search_num

In [32]:
total = 0
ct = 0
for i in range(100) : 
    print('epoch', i)
    for video_id in video_ids:
        print(video_id)
        tss = df_test_dr_valid.loc[(df_test_dr_valid['video_id'] == video_id)]['middle_frame_timestamp'].to_list()
        print(tss)

        as_metric = 500
        while (as_metric == 500) :
            as_metric = find_action(tss, video_id, model.get_init_hidden_states())

        total = total + as_metric
        ct = ct + 1
#         print('avg', total/ct)
print('avg', total/ct)

epoch 0
7YpF6DntOYw
[1117, 1117, 1120, 1120, 1192, 1195, 1198, 1201]
975.0
tensor([[[0.1313]]], grad_fn=<AddBackward0>)
975.0
tensor([[[0.1288]]], grad_fn=<AddBackward0>)
975.0
tensor([[[0.1394]]], grad_fn=<AddBackward0>)
975.0
tensor([[[0.1415]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.1445]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.1450]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.1436]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.1380]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0654]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0659]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0430]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0687]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0611]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0836]]], grad_fn=<AddBackward0>)
976.0
tensor([[[0.0967]]], grad_fn=<AddBackward0>)
977.0
tensor([[[0.1249]]], grad_fn=<AddBackward0>)
977.0
tensor([[[0.1283]]], grad_fn=<AddBackward0>)
977.0
tensor([[[0.1070]]], grad_fn=<AddBackward0>)
977.0
tensor(

KeyboardInterrupt: ignored